In [ ]:
# https://heartbeat.fritz.ai/guide-to-restricted-boltzmann-machines-using-pytorch-ee50d1ed21a8
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [9]:
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip -o ml-latest-small.zip -d ./data
!unzip -o ml-latest-small.zip -d ./data

--2019-08-27 17:50:33--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org... 128.101.65.152
Connecting to files.grouplens.org|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: 'ml-latest-small.zip.2'

ml-latest-small.zip 100%[=====================>] 955.28K   367KB/s   in 2.6s   

2019-08-27 17:50:36 (367 KB/s) - 'ml-latest-small.zip.2' saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ./data/ml-latest-small/
  inflating: ./data/ml-latest-small/links.csv  
  inflating: ./data/ml-latest-small/tags.csv  
  inflating: ./data/ml-latest-small/ratings.csv  
  inflating: ./data/ml-latest-small/README.txt  
  inflating: ./data/ml-latest-small/movies.csv  
Archive:  ml-latest-small.zip
  inflating: ./data/ml-latest-small/links.csv  
  inflating: ./data/ml-latest-small/tags.csv  
  inflating: ./data/ml-latest-small/ratings.csv  
  inflating: ./data/ml-la

In [24]:
movies = pd.read_csv('data/ml-latest-small/movies.csv', sep = ',', header = None, engine = 'python')
users = pd.read_csv('data/ml-latest-small/links.csv', sep = ',', header = None, engine = 'python')
ratings = pd.read_csv('data/ml-latest-small/ratings.csv', sep = ',', header = None, engine = 'python')

In [26]:
training_dataset = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_dataset = np.array(training_set, dtype = 'int')
test_dataset = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_dataset = np.array(test_dataset, dtype = 'int') 

In [ ]:
no_users = int(max(max(training_dataset[:,0]), max(test_dataset[:,0])))
no_movies = int(max(max(training_dataset[:,1]), max(test_dataset[:,1])))

In [ ]:
def convert_dataset(data):
    converted_data = []
    for id_users in range(1, no_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        movie_ratings = np.zeros(no_movies)
        ratings[id_movies - 1] = id_ratings
        converted_data.append(list(movie_ratings))
    return converted_data

In [ ]:
training_dataset = convert_dataset(training_dataset)
test_dataset = convert_dataset(test_dataset)

In [ ]:
training_dataset = torch.FloatTensor(training_dataset)
test_dataset = torch.FloatTensor(test_dataset)

In [ ]:
training_dataset[training_dataset == 0] = -1
training_dataset[training_dataset == 1] = 0
training_dataset[training_dataset == 2] = 0
training_dataset[training_dataset >= 3] = 1
test_dataset[test_dataset == 0] = -1
test_dataset[test_dataset == 1] = 0
test_dataset[test_dataset == 2] = 0
test_dataset[test_dataset >= 3] = 1

In [ ]:
class RBM():
    def __init__(self, num_visible_nodes, num_hidden_nodes):
        self.W = torch.randn(num_hidden_nodes, num_visible_nodes)
        self.a = torch.randn(1, num_hidden_nodes)
        self.b = torch.randn(1, num_visible_nodes)

    def sample_hidden_nodes(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)

    def sample_visible_nodes(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)

    def train(self, v0, vk, ph0, phk):
        self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

In [ ]:
num_visible_nodes = len(training_dataset[0])
num_hidden_nodes = 200
batch_size = 100
rbm = RBM(num_visible_nodes, num_hidden_nodes)

In [ ]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_dataset[id_user:id_user+batch_size]
        v0 = training_dataset[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_hidden_nodes(v0)
        for k in range(10):
            _,hk = rbm.sample_hidden_nodes(vk)
            _,vk = rbm.sample_visible_nodes(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_hidden_nodes(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

In [ ]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_dataset[id_user:id_user+1]
    vt = test_dataset[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_hidden_nodes(v)
        _,v = rbm.sample_visible_nodes(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))